*(Please note that the contents of this notebook are identical to session_02_bathymetry.ipynb, which we did not get to cover during session 2)*

# Session 3 Interpolation using SciPy

In this notebook we will import a set of land surface heights which were measured in the field using a levelling device. The ultimate aim is to use this data to develop a relationship between the water level in the farm dam and the water surface and water volume. The latter two are needed in the water balance calculations. In order to establish this relation it is necesssary to interpolate the (irregularly spaced) data points to a regular grid. This requires some pre-processing of the data, which will be the starting point of this exercise.

## Data pre-processing

The following packages must be imported first

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import pandas as pd
from scipy.interpolate import griddata

In order to save the data as a shapefile, it is necessary to define a reference coordinate system. This is done using GeoPandas, which relies on the pyproj package. However, the latter may not always be able to find its own database and just to be sure we won't encounter any errors later on, we use the function `set_data_dir` (which was added to pyproj especially for this purpose). Note that the directory name below differs for each computer.

In [ ]:
from pyproj.datadir import set_data_dir

set_data_dir("c:/Users/vince/anaconda3/envs/geopandas_env/Library/share/proj/")
# set_data_dir("c:/Users/VincentPost/anaconda3/envs/geopandas_env/Library/share/proj/")

The data in the Excel file 'dam_bathymetry.xlsx' can be read using Pandas' `read_excel` function. Several keyword arguments are passed to read only the relevant rows and columns.

In [ ]:
df = pd.read_excel(
    "data/dam_bathymetry.xlsx",
    skiprows=5,
    usecols="B,E,F",
    header=None,
    names=["z", "easting", "northing"],
)

Once loaded as a DataFrame, it is easy to add the spatial attributes and create a GeoDataFrame.

In [ ]:
utm_coordinates = gpd.points_from_xy(df["easting"], df["northing"])
gdf = gpd.GeoDataFrame(
    data=df["z"], 
    geometry=utm_coordinates, 
    crs="epsg:32754",
)

Saving the GeoDataFrame as an ESRI Shapefile allows it to be inspected in a GIS program

In [ ]:
Path("data/dam_bathymetry").mkdir(exist_ok=True)
gdf.to_file("data/dam_bathymetry/dam_bathymetry.shp")

To get a better interpolation result, a polygon was drawn around the data points and saved as an ESRI Shapefile, which can be imported using `read_file`

In [ ]:
gdf_h = gpd.read_file("data/helper_poly/helper_poly.shp")
print(gdf_h)

The vertices of the polygon can be extracted from the GeoDataFrame. The result is a list variable `poly_vertices`, which contains two arrays, one with the eastings and one with the northings of the vertices.

In [ ]:
helper_poly = gdf_h['geometry'][0]
poly_vertices = helper_poly.exterior.xy

The vertices can then be turned into helper points for the interpolation. The code cell below stores them in a DataFrame with the same structure as `df`, the DataFrame with the actual measurements. A surface elevation (`z`) of -0.2 m with respect to the local reference level is assumed.

In [ ]:
dfh = pd.DataFrame(
    data={
        "easting": poly_vertices[0],
        "northing": poly_vertices[1],
        "z": -0.20,
    }
)

The `concat` function combines the DataFrames `df` and `dfh`. The result is stored in `df`, so the original `df` (containing only the measurements) is overwritten. The new `df` contains the data that form the basis for the interpolation.

In [ ]:
df = pd.concat((df, dfh))

The regular grid to which to interpolate the land surface heights is created in the code cell below. A grid resolution of 0.5 m is used and grid points are placed in both the x and y direction between the respective minimum and maximum coordinate values using the `arange` function.

In [ ]:
xmin, xmax = df['easting'].min(), df['easting'].max()
ymin, ymax = df['northing'].min(), df['northing'].max()
dx = dy = 0.5
xi = np.arange(xmin - 2, xmax + 2, dx)
yi = np.arange(ymin - 2, ymax + 2, dy)

To check if the grid encompasses the entire set of data points we can plot them in a figure. In order to plot the grid we create two two-dimensional arrays `X` and `Y` using `meshgrid`. This function returns two arrays of which the corresponding elements form the coordinate pairs of the points.

In [ ]:
X, Y = np.meshgrid(xi, yi)
fig = plt.figure()
plt.plot(X, Y, 'k.') # Plot grid as black dots
plt.plot(df['easting'], df['northing'], 'rs'); # Plot data points as red squares

***Exercise 1***: The grid points are so closely spaced that they show up as a black rectangle instead of individual dots. Use array slicing to plot only every second grid point.

In [ ]:
# Type your answer here

The next cell uses the `griddata` function to interpolate the surface heights onto the regular grid. There are different methods to choose from and to decide which method gives the best result, a `for` loop is created that loops over the method names stored in a list. Two figures with both three subplots are created and in each of the subplots the result of the interpolation will be visualized: as a filled contour map in the one figure, and as a 3D wireframe in the other.

In [ ]:
cl_levels = np.arange(-2.75, 0, 0.25) # Contour levels to plot

fig3d = plt.figure()
fig_contours, axs_contours = plt.subplots(ncols=3)

for i, method in enumerate(["nearest", "cubic", "linear"]):
    zi = griddata(
        (df['easting'], df['northing']), 
        df['z'], 
        (xi[None, :], yi[:, None]), 
        method=method,
    )

    ax = axs_contours[i]
    cs = ax.contourf(X, Y, zi, cl_levels)

    ax = fig3d.add_subplot(1, 3, i+1, projection='3d')
    ax.plot_wireframe(X, Y, zi, rstride=4, cstride=4)

    # Measurements
    ax.scatter(df['easting'], df['northing'], df['z'], color='k')
    
    # Helper points
    idx = df['z'] == -0.20
    ax.scatter(df.loc[idx, 'easting'], df.loc[idx, 'northing'], df.loc[idx, 'z'], color='r')

Once we know the land surface height at every grid point we can use this result to develop the relationship between water level and water surface area and volume. The next set of code cells provides a rather crude, but easy to implement, method: An array with water levels ranging between the minimum interpolated land surface height and -0.2 m at 0.01 m increments is created first

In [ ]:
z_min = int(np.nanmin(zi) * 100) / 100
wls = np.arange(-0.2, z_min, -0.01)

Then a `for` loop steps over each water level defined in the array `wls`. It calculates the water depth `wd` at each grid point and it is checked at which values in the grid the water level is greater than zero. The command `np.sum(idx)` sums the number of cells in which the water depth is greater than zero because a `True` value in a boolean array represents a value of 1 (conversely `False` is zero). The total volume (`V`) is the sum of the volumes of the water columns of the individual grid cells.

In [ ]:
A = []
V = []
for wl in wls:
    wd = wl - zi
    idx = wd > 0
    A.append(np.sum(idx) * dx ** 2)
    V.append(np.sum(wd[idx] * dx ** 2))

When the surface areas and water volumes are known for each water level value in `wls`, a polynomial can be fitted to the data so that we obtain a mathematical relationship that allows us to convert the measured water levels to a corresponding water surface area and volume (which will be done when we consider the water balance during the following sessions and therefore the coefficients are stored in text files for later use). The `polyfit` function fits the polynomial and the resulting coefficients can be used to define the polynomial function itself, which is used in the next code cell to visually check if the fitted line passes through the actual data.

In [ ]:
# Area
p_coef_A = np.polyfit(wls, A, 6)
p_func_A = np.poly1d(p_coef_A)

# Volume
p_coef_V = np.polyfit(wls, V, 4)
p_func_V = np.poly1d(p_coef_V)

np.savetxt(f"p_coef_V_{method}.dat", p_coef_V)
np.savetxt(f"p_coef_A_{method}.dat", p_coef_A)

In [ ]:
fig_lines, axs = plt.subplots(ncols=2)

# Area
ax = axs[0]
ax.plot(wls, A, 'C0.')
ax.plot(wls, p_func_A(wls), 'k')
ax.set_ylabel("Area (m$^2$)")

# Volume
ax = axs[1]
ax.plot(wls, V, 'C0.')
ax.plot(wls, p_func_V(wls), 'k')
ax.set_ylabel("Volume (m$^3$)")

# Set some properties for both graphs
for ax in axs:
    ax.set_xlabel("Water level (m)")
    ax.grid(ls=":")

## Bonus material: Converting the matplotlib contours to a Shapefile

The code cells below delves into the contour lines (of the last figure that was created) and pulls out their vertices. It's a rather complex piece of code that uses the Matplotlib object model. It is provided here without additional explanation (we can revisit this in session 8 upon request) but the final result is an ESRI Shapefile that contains the same contour as the third of the three figures shown above. This provides a way to also show the contours in a GIS.

In [ ]:
import shapely.geometry as shg

lvl_lookup = dict(zip(cs.levels, cs.collections))

level_list = []
poly_list = []
for k, v in lvl_lookup.items():
    level_list.append(k)
    xys = v.get_paths()
    xy = xys[0].to_polygons()[0]
    poly_list.append(shg.Polygon(xy))

In [ ]:

Path("data/interpolated_contours").mkdir(exist_ok=True)
gdf = gpd.GeoDataFrame(data={'level': level_list}, geometry=poly_list, crs="epsg:32754")
gdf.to_file(f"data/interpolated_contours/interpolated_contours_{method}.shp")